In [2]:
#import necessary libraries
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import os

In [3]:
#import tensorflow and keras
import tensorflow as tf
from tensorflow import keras

In [4]:
#assign image directory to variable
train_dir = r'data\train'
test_dir = r'data\test'
validation_dir = r'data\validation'

In [5]:
#preprocess images
#note that the images are not all the same size
# and they are colored
from tensorflow.keras.preprocessing.image import ImageDataGenerator